In [3]:
import os, sys
print(os.getcwd())  # 현재 작업 디렉토리 출력
current_dir = os.path.dirname(os.path.abspath("/mnt/batch/tasks/shared/LS_root/mounts/clusters/b001-dl/code/Users/5b001/DL3_20241006/common")  # 현재 파일의 디렉토리 경로 가져오기
print(current_dir)
os.chdir(current_dir)  # 현재 디렉토리로 변경

import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist  # 숫자 데이터를 다운로드
from common.util import smooth_curve
from common.multi_layer_net import MultilayerNet
from common.optimizer import SGD  # Stochastic Gradient Descent Optimizer 사용

# 0. MNIST 데이터 읽기 ==================
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)  # MNIST 데이터셋 로드
train_size = x_train.shape[0]  # 학습 데이터 크기 정의
batch_size = 128  # 미니배치 크기 설정
max_iterations = 2000  # 최대 반복 횟수 설정

# 1. 실험 설정 ===================
weight_init_types = {'std=0.01': 0.01, 'Xavier': 'sigmoid', 'He': 'relu'}  # Weight 초기값 종류별 지정
optimizer = SGD(lr=0.01)  # 학습률이 0.01인 SGD Optimizer 사용

networks = {}  # 신경망 딕셔너리 생성
train_loss = {}  # 학습 손실값을 저장할 딕셔너리 생성

for key, weight_type in weight_init_types.items():  # Weight 초기화 방법별로 신경망 생성
    networks[key] = MultilayerNet(input_size=784, hidden_size_list=[100, 100, 100, 100],
                                  output_size=10, weight_init_std=weight_type)

train_loss[key] = []  # 초기 손실값 저장을 위한 리스트 생성

# 2. 훈련 시작 ===================
for i in range(max_iterations):
    batch_mask = np.random.choice(train_size, batch_size)  # 랜덤으로 배치 크기만큼 데이터 샘플 추출
    x_batch = x_train[batch_mask]  # 학습용 배치 데이터
    t_batch = t_train[batch_mask]  # 학습용 배치 레이블

    for key in weight_init_types.keys():  # 네트워크별로 기울기를 계산하고 가중치를 업데이트
        grads = networks[key].gradient(x_batch, t_batch)
        optimizer.update(networks[key].params, grads)

        loss = networks[key].loss(x_batch, t_batch)  # 배치 데이터를 통해 손실값 계산
        train_loss[key].append(loss)  # 계산된 손실값을 저장

    if i % 100 == 0:  # 100회마다 학습 경과 출력
        print("===========" + "iteration:" + str(i) + "===========")
        for key in weight_init_types.keys():  # Weight 초기화 방법별 손실값 출력
            loss = networks[key].loss(x_batch, t_batch)
            print(key + ':' + str(loss))

# 3. 그래프 그리기 =================
markers = {'std=0.01': 'o', 'Xavier': 's', 'He': 'D'}  # 그래프에 사용할 마커 지정
x = np.arange(max_iterations)  # x축에 사용할 값들 (반복 횟수)

for key in weight_init_types.keys():
    plt.plot(x, smooth_curve(train_loss[key]), marker=markers[key], markevery=100, label=key)  # 손실값 그래프

plt.xlabel("iterations")  # x축 레이블
plt.ylabel("loss")  # y축 레이블
plt.ylim(0, 2.5)  # y축 범위 설정
plt.legend()  # 범례 표시
plt.show()  # 그래프 출력


/mnt/batch/tasks/shared/LS_root/mounts/clusters/b001-dl/code/Users/5b001/DL3_20241006
/mnt/batch/tasks/shared/LS_root/mounts/clusters/b001-dl/code/Users/5b001/DL3_20241006


ImportError: cannot import name 'MultilayerNet' from 'common.multi_layer_net' (/mnt/batch/tasks/shared/LS_root/mounts/clusters/b001-dl/code/Users/5b001/DL3_20241006/common/multi_layer_net.py)